In [406]:
import pandas as pd
%matplotlib inline
from matplotlib import pyplot as plt
import seaborn as sns
import numpy as np
from sklearn.preprocessing import StandardScaler, RobustScaler, MinMaxScaler
from sklearn.model_selection import train_test_split
import pickle

from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.pipeline import FeatureUnion
from sklearn import set_config; set_config(display='diagram')
from sklearn.preprocessing import OneHotEncoder
# Algorithms
from sklearn.linear_model import LinearRegression

from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV
from sklearn.inspection import permutation_importance
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')

In [407]:
from sklearn import metrics
from sklearn.metrics import mean_squared_error

def rmse(y, y_):
    return np.sqrt(np.mean((y-y_)**2))

def mae(y, y_):
    return np.mean(np.abs(y-y_))

###  Modèles de régression à sorties multiples

Pour cette partie on va tester des algorithmes lineaires et non lineaires

In [408]:
data=pd.read_csv("cleaning_data.csv")
data=data.drop("Unnamed: 0",axis=1)
data.NumberofBuildings=data.NumberofBuildings.astype("int64")

In [409]:
data.head()

,BuildingType,PrimaryPropertyType,Neighborhood,NumberofBuildings,NumberofFloors,PropertyGFATotal,LargestPropertyUseType,SiteEUI(kBtu/sf),SiteEnergyUse(kBtu),TotalGHGEmissions,GHGEmissionsIntensity,BuildingAge,TotalUseTypeNumber,GFABuildingRate,GFAParkingRate,GFAPerBuilding,GFAPerFloor,harvesine_distance
0,NonResidential,Hotel,DOWNTOWN,1,12,88434,Hotel,81.699997,7226362.5,249.98,2.83,89,1,1.00000,0.00000,88434.0,7369.500,0.496782
1,NonResidential,Hotel,DOWNTOWN,1,11,103566,Hotel,94.800003,8387933.0,295.86,2.86,20,3,0.85455,0.14545,103566.0,9415.091,0.488726
2,NonResidential,Hotel,DOWNTOWN,1,41,956110,Hotel,96.000000,72587024.0,2089.28,2.19,47,1,0.79425,0.20575,956110.0,23319.756,0.602378
3,NonResidential,Hotel,DOWNTOWN,1,10,61320,Hotel,110.800003,6794584.0,286.43,4.67,90,1,1.00000,0.00000,61320.0,6132.000,0.586253
4,NonResidential,Hotel,DOWNTOWN,1,18,175580,Hotel,114.800003,14172606.0,505.01,2.88,36,3,0.64688,0.35312,175580.0,9754.444,0.650801


In [410]:
#enlever les lignes qui ont manque de données suivant les deux tragets
data=data[data['SiteEnergyUse(kBtu)']!=0]
data=data[data['TotalGHGEmissions']!=0]

In [411]:
data.shape

(3193, 18)

In [412]:
Y=data[['TotalGHGEmissions','SiteEnergyUse(kBtu)']]
X=data.drop(['TotalGHGEmissions','SiteEnergyUse(kBtu)'],axis=1)

In [413]:
X_train, X_test, y_train, y_test = train_test_split(X,Y,test_size=0.25, random_state= 2022)

In [414]:
num_columns = X.columns[X.dtypes != 'object']
cat_columns = X.columns[X.dtypes == 'object']

In [415]:
#pipeline
categorical_transformer = Pipeline(steps=[
    ('onehot', OneHotEncoder(handle_unknown='ignore')),
])

numeric_transformer = Pipeline(steps=[
   ('scaler',RobustScaler()),
   ])
from sklearn.compose import make_column_transformer
preprocessor = make_column_transformer((RobustScaler(),num_columns),
(OneHotEncoder(handle_unknown = 'ignore'),cat_columns ))

In [416]:
from sklearn.dummy import DummyRegressor
dummy_pipe = Pipeline([
        ('transformer', preprocessor),
        ('dummy', DummyRegressor(strategy='median')),
    ])

dummy_pipe.fit(X_train,y_train)

Pipeline(steps=[('transformer',
                 ColumnTransformer(transformers=[('robustscaler',
                                                  RobustScaler(),
                                                  Index(['NumberofBuildings', 'NumberofFloors', 'PropertyGFATotal',
       'SiteEUI(kBtu/sf)', 'GHGEmissionsIntensity', 'BuildingAge',
       'TotalUseTypeNumber', 'GFABuildingRate', 'GFAParkingRate',
       'GFAPerBuilding', 'GFAPerFloor', 'harvesine_distance'],
      dtype='object')),
                                                 ('onehotencoder',
                                                  OneHotEncoder(handle_unknown='ignore'),
                                                  Index(['BuildingType', 'PrimaryPropertyType', 'Neighborhood',
       'LargestPropertyUseType'],
      dtype='object'))])),
                ('dummy', DummyRegressor(strategy='median'))])

In [417]:
y_pred_dummy=dummy_pipe.predict(X_test)
print("R2_GHG: ", metrics.r2_score(y_test.iloc[:,0],y_pred_dummy[:,0]))
print("R2_enrgy: ", metrics.r2_score(y_test.iloc[:,1], y_pred_dummy[:,1]))
print("MAE: ", mae(y_test,y_pred_dummy))
# score_dummy = mean_squared_error(y_test, y_pred_dummy)
# print("mse:",score_dummy )
# print("rmse:",rmse(y_test, y_pred_dummy))

R2_GHG:  -0.021847528295714236
R2_enrgy:  -0.036025186983595026
MAE:  TotalGHGEmissions      1.171928e+02
SiteEnergyUse(kBtu)    4.755525e+06
dtype: float64


In [418]:
lr_pipe = Pipeline([
       ('transformer', preprocessor),
        ('lr',  LinearRegression()),
    ])

lr_pipe.fit(X_train,y_train)
y_pred = lr_pipe.predict(X_test)

In [419]:
from sklearn import metrics
print("R2_GHG: ", metrics.r2_score(y_test.iloc[:,0],y_pred [:,0]))
print("R2_enrgy: ", metrics.r2_score(y_test.iloc[:,1], y_pred [:,1]))
print("MAE: ", mae(y_test, y_pred))
#print("rmse:",rmse(y_test, y_pred))

R2_GHG:  0.3346906823699667
R2_enrgy:  0.3236930814466882
MAE:  TotalGHGEmissions      1.007563e+02
SiteEnergyUse(kBtu)    3.128224e+06
dtype: float64


In [420]:
from sklearn.linear_model import ElasticNet
elasticnet_pipe = Pipeline([
         ('transformer', preprocessor),
        ('elastic_net',  ElasticNet()),
    ])

elasticnet_pipe.fit(X_train, y_train)

Pipeline(steps=[('transformer',
                 ColumnTransformer(transformers=[('robustscaler',
                                                  RobustScaler(),
                                                  Index(['NumberofBuildings', 'NumberofFloors', 'PropertyGFATotal',
       'SiteEUI(kBtu/sf)', 'GHGEmissionsIntensity', 'BuildingAge',
       'TotalUseTypeNumber', 'GFABuildingRate', 'GFAParkingRate',
       'GFAPerBuilding', 'GFAPerFloor', 'harvesine_distance'],
      dtype='object')),
                                                 ('onehotencoder',
                                                  OneHotEncoder(handle_unknown='ignore'),
                                                  Index(['BuildingType', 'PrimaryPropertyType', 'Neighborhood',
       'LargestPropertyUseType'],
      dtype='object'))])),
                ('elastic_net', ElasticNet())])

In [421]:
y_pred_elastic_net= elasticnet_pipe.predict(X_test)

print("R2_GHG: ", metrics.r2_score(y_test.iloc[:,0],y_pred_elastic_net[:,0]))
print("R2_enrgy: ", metrics.r2_score(y_test.iloc[:,1], y_pred_elastic_net[:,1]))
print("MAE:",mae(y_test,y_pred_elastic_net))

R2_GHG:  0.3619757336979853
R2_enrgy:  0.38717006714625535
MAE: TotalGHGEmissions      1.037634e+02
SiteEnergyUse(kBtu)    3.348732e+06
dtype: float64


In [422]:
from sklearn import tree
from sklearn.tree import DecisionTreeRegressor
tree_pipe = Pipeline([
        ('transformer', preprocessor),
        ('model_tree',  DecisionTreeRegressor()),
    ])
 
tree_pipe.fit(X_train,y_train)
 
y_pred_tree = tree_pipe.predict(X_test)
 
print("R2_GHG: ", metrics.r2_score(y_test.iloc[:,0],y_pred_tree[:,0]))
print("R2_enrgy: ", metrics.r2_score(y_test.iloc[:,1], y_pred_tree[:,1]))
print("MAE: ", mae(y_test, y_pred))

R2_GHG:  0.3083738455526802
R2_enrgy:  0.3859845822466126
MAE:  TotalGHGEmissions      1.007563e+02
SiteEnergyUse(kBtu)    3.128224e+06
dtype: float64


### RandomForestRegressor


In [423]:
from sklearn.ensemble import RandomForestRegressor
rf_pipeline = Pipeline(
    [
       ('preprocessing', preprocessor),
        ("clf", RandomForestRegressor(n_jobs=-1, random_state=0)),
    ]
)
rf_pipeline.fit(X_train, y_train)

Pipeline(steps=[('preprocessing',
                 ColumnTransformer(transformers=[('robustscaler',
                                                  RobustScaler(),
                                                  Index(['NumberofBuildings', 'NumberofFloors', 'PropertyGFATotal',
       'SiteEUI(kBtu/sf)', 'GHGEmissionsIntensity', 'BuildingAge',
       'TotalUseTypeNumber', 'GFABuildingRate', 'GFAParkingRate',
       'GFAPerBuilding', 'GFAPerFloor', 'harvesine_distance'],
      dtype='object')),
                                                 ('onehotencoder',
                                                  OneHotEncoder(handle_unknown='ignore'),
                                                  Index(['BuildingType', 'PrimaryPropertyType', 'Neighborhood',
       'LargestPropertyUseType'],
      dtype='object'))])),
                ('clf', RandomForestRegressor(n_jobs=-1, random_state=0))])

In [424]:
#rf_pipeline.get_params().keys()

In [425]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3193 entries, 0 to 3212
Data columns (total 18 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   BuildingType            3193 non-null   object 
 1   PrimaryPropertyType     3193 non-null   object 
 2   Neighborhood            3193 non-null   object 
 3   NumberofBuildings       3193 non-null   int64  
 4   NumberofFloors          3193 non-null   int64  
 5   PropertyGFATotal        3193 non-null   int64  
 6   LargestPropertyUseType  3193 non-null   object 
 7   SiteEUI(kBtu/sf)        3193 non-null   float64
 8   SiteEnergyUse(kBtu)     3193 non-null   float64
 9   TotalGHGEmissions       3193 non-null   float64
 10  GHGEmissionsIntensity   3193 non-null   float64
 11  BuildingAge             3193 non-null   int64  
 12  TotalUseTypeNumber      3193 non-null   int64  
 13  GFABuildingRate         3193 non-null   float64
 14  GFAParkingRate          3193 non-null   

In [426]:
parameters = {
    'clf__n_estimators' : [10,50,100,300], #nombre d'arbres de décision
    'clf__min_samples_leaf' : [1,3,5], #nombre de feuilles minimales dans un noeud
    'clf__max_features': ['auto', 'sqrt'] #nombre de features observées pour chaque arbre
}

RandomForestmodel_search = GridSearchCV(rf_pipeline,
                               param_grid = parameters,
                              verbose=2,
                               cv=5)

RandomForestmodel_search.fit(X_train, y_train)

Fitting 5 folds for each of 24 candidates, totalling 120 fits
[CV] END clf__max_features=auto, clf__min_samples_leaf=1, clf__n_estimators=10; total time=   2.2s
[CV] END clf__max_features=auto, clf__min_samples_leaf=1, clf__n_estimators=10; total time=   0.2s
[CV] END clf__max_features=auto, clf__min_samples_leaf=1, clf__n_estimators=10; total time=   0.2s
[CV] END clf__max_features=auto, clf__min_samples_leaf=1, clf__n_estimators=10; total time=   0.3s
[CV] END clf__max_features=auto, clf__min_samples_leaf=1, clf__n_estimators=10; total time=   0.2s
[CV] END clf__max_features=auto, clf__min_samples_leaf=1, clf__n_estimators=50; total time=   1.1s
[CV] END clf__max_features=auto, clf__min_samples_leaf=1, clf__n_estimators=50; total time=   1.1s
[CV] END clf__max_features=auto, clf__min_samples_leaf=1, clf__n_estimators=50; total time=   1.1s
[CV] END clf__max_features=auto, clf__min_samples_leaf=1, clf__n_estimators=50; total time=   1.1s
[CV] END clf__max_features=auto, clf__min_sampl

GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('preprocessing',
                                        ColumnTransformer(transformers=[('robustscaler',
                                                                         RobustScaler(),
                                                                         Index(['NumberofBuildings', 'NumberofFloors', 'PropertyGFATotal',
       'SiteEUI(kBtu/sf)', 'GHGEmissionsIntensity', 'BuildingAge',
       'TotalUseTypeNumber', 'GFABuildingRate', 'GFAParkingRate',
       'GFAPerBuilding', 'GFAPerFloor', 'harvesine_distance'],
      dtype='object')),
                                                                        ('onehotencoder',
                                                                         OneHotEncoder(handle_unknown='ignore'),
                                                                         Index(['BuildingType', 'PrimaryPropertyType', 'Neighborhood',
       'LargestPropertyUseType'],
      dtype='object'))])),
                                       ('clf',
                                        RandomForestRegressor(n_jobs=-1,
                                                              random_state=0))]),
             param_grid={'clf__max_features': ['auto', 'sqrt'],
                         'clf__min_samples_leaf': [1, 3, 5],
                         'clf__n_estimators': [10, 50, 100, 300]},
             verbose=2)

In [427]:
RandomForestmodel_search.best_params_

{'clf__max_features': 'sqrt',
 'clf__min_samples_leaf': 1,
 'clf__n_estimators': 300}

In [428]:
RandomForestmodel_search.best_params_
preds= RandomForestmodel_search.predict(X_test)

In [429]:
print("R2_GHG: ", metrics.r2_score(y_test.iloc[:,0], preds[:,0]))
print("R2_enrgy: ", metrics.r2_score(y_test.iloc[:,1], preds[:,1]))
print("MAE: ", mae(y_test, preds))

R2_GHG:  0.25221188861030475
R2_enrgy:  0.3397238779473696
MAE:  TotalGHGEmissions      6.161426e+01
SiteEnergyUse(kBtu)    2.136906e+06
dtype: float64


In [430]:
best_model=RandomForestmodel_search.best_estimator_



In [482]:
#rf_pipeline.get_feature_names_out()

In [483]:
#rf_pipeline.named_steps['clf'].feature_importances_

In [432]:
data.to_csv('data_model.csv')

In [444]:
with open('model.pkl', 'wb') as fileWriteStream:
    pickle.dump(best_model, fileWriteStream)

## Appliquer log pour les deux targets avec TransformedTargetRegressor

In [434]:

# Y_log=np.log(data[['TotalGHGEmissions','SiteEnergyUse(kBtu)']])
# Y_log.head()

In [435]:
#model_log.get_params().keys()

In [457]:
model_log.named_steps

{'preprocessing': ColumnTransformer(transformers=[('robustscaler', RobustScaler(),
                                  Index(['NumberofBuildings', 'NumberofFloors', 'PropertyGFATotal',
        'SiteEUI(kBtu/sf)', 'GHGEmissionsIntensity', 'BuildingAge',
        'TotalUseTypeNumber', 'GFABuildingRate', 'GFAParkingRate',
        'GFAPerBuilding', 'GFAPerFloor', 'harvesine_distance'],
       dtype='object')),
                                 ('onehotencoder',
                                  OneHotEncoder(handle_unknown='ignore'),
                                  Index(['BuildingType', 'PrimaryPropertyType', 'Neighborhood',
        'LargestPropertyUseType'],
       dtype='object'))]),
 'model': TransformedTargetRegressor(func=<ufunc 'log1p'>, inverse_func=<ufunc 'expm1'>,
                            regressor=RandomForestRegressor(n_jobs=-1,
                                                            random_state=0))}

In [459]:
model_log.named_steps['model'].

NotFittedError: This RandomForestRegressor instance is not fitted yet. Call 'fit' with appropriate arguments before using this estimator.

In [437]:
from sklearn.compose import TransformedTargetRegressor
model_log = Pipeline([
    ('preprocessing', preprocessor),
('model', TransformedTargetRegressor(
                                    regressor=RandomForestRegressor(n_jobs=-1, random_state=0),
                                     func=np.log1p, 
                                     inverse_func=np.expm1))
])



model_log.fit(X_train, y_train)
parameters = {
    'model__regressor__n_estimators': [10,50,100,300], #nombre d'arbres de décision
   'model__regressor__min_samples_leaf' : [1,3,5], #nombre de feuilles minimales dans un noeud
    #'model__regressor__min_samples_leaf': ['auto', 'sqrt'] #nombre de features observées pour chaque arbre
}

RandomForestmodel_search_log = GridSearchCV(model_log,
                               param_grid = parameters,
                              verbose=2,
                               cv=5)

RandomForestmodel_search_log.fit(X_train, y_train)

Fitting 5 folds for each of 12 candidates, totalling 60 fits
[CV] END model__regressor__min_samples_leaf=1, model__regressor__n_estimators=10; total time=   0.2s
[CV] END model__regressor__min_samples_leaf=1, model__regressor__n_estimators=10; total time=   0.2s
[CV] END model__regressor__min_samples_leaf=1, model__regressor__n_estimators=10; total time=   0.2s
[CV] END model__regressor__min_samples_leaf=1, model__regressor__n_estimators=10; total time=   0.2s
[CV] END model__regressor__min_samples_leaf=1, model__regressor__n_estimators=10; total time=   0.2s
[CV] END model__regressor__min_samples_leaf=1, model__regressor__n_estimators=50; total time=   1.1s
[CV] END model__regressor__min_samples_leaf=1, model__regressor__n_estimators=50; total time=   1.1s
[CV] END model__regressor__min_samples_leaf=1, model__regressor__n_estimators=50; total time=   1.1s
[CV] END model__regressor__min_samples_leaf=1, model__regressor__n_estimators=50; total time=   1.3s
[CV] END model__regressor__min

GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('preprocessing',
                                        ColumnTransformer(transformers=[('robustscaler',
                                                                         RobustScaler(),
                                                                         Index(['NumberofBuildings', 'NumberofFloors', 'PropertyGFATotal',
       'SiteEUI(kBtu/sf)', 'GHGEmissionsIntensity', 'BuildingAge',
       'TotalUseTypeNumber', 'GFABuildingRate', 'GFAParkingRate',
       'GFAPerBuilding', 'GFAPerFloor', 'harvesine_distance...
                                                                         Index(['BuildingType', 'PrimaryPropertyType', 'Neighborhood',
       'LargestPropertyUseType'],
      dtype='object'))])),
                                       ('model',
                                        TransformedTargetRegressor(func=<ufunc 'log1p'>,
                                                                   inverse_func=<ufunc 'expm1'>,
                                                                   regressor=RandomForestRegressor(n_jobs=-1,
                                                                                                   random_state=0)))]),
             param_grid={'model__regressor__min_samples_leaf': [1, 3, 5],
                         'model__regressor__n_estimators': [10, 50, 100, 300]},
             verbose=2)

In [438]:
RandomForestmodel_search_log.best_params_
preds_log= RandomForestmodel_search_log.predict(X_test)

In [439]:
print("R2_GHG: ", metrics.r2_score(y_test.iloc[:,0], preds_log[:,0]))
print("R2_enrgy: ", metrics.r2_score(y_test.iloc[:,1], preds_log[:,1]))
print("MAE: ", mae(y_test, preds_log))

R2_GHG:  0.4918696615599515
R2_enrgy:  0.5016346741252444
MAE:  TotalGHGEmissions      3.074988e+01
SiteEnergyUse(kBtu)    1.666366e+06
dtype: float64


In [454]:
log_model=RandomForestmodel_search_log.best_estimator_

<bound method Pipeline.get_feature_names_out of Pipeline(steps=[('preprocessing',
                 ColumnTransformer(transformers=[('robustscaler',
                                                  RobustScaler(),
                                                  Index(['NumberofBuildings', 'NumberofFloors', 'PropertyGFATotal',
       'SiteEUI(kBtu/sf)', 'GHGEmissionsIntensity', 'BuildingAge',
       'TotalUseTypeNumber', 'GFABuildingRate', 'GFAParkingRate',
       'GFAPerBuilding', 'GFAPerFloor', 'harvesine_distance'],
      dtype='object')),
                                                 ('onehotencoder',
                                                  OneHotEncoder(handle_unknown='ignore'),
                                                  Index(['BuildingType', 'PrimaryPropertyType', 'Neighborhood',
       'LargestPropertyUseType'],
      dtype='object'))])),
                ('model',
                 TransformedTargetRegressor(func=<ufunc 'log1p'>,
                            

In [452]:
compar = pd.DataFrame([y_test.iloc[:,0], log_model.predict(X_test)[:,0],y_test.iloc[:,1],log_model.predict(X_test)[:,1]]).T
compar.columns = ['obsGHG','pred1','obs_eng','pred2']
compar['diff1'] = compar.obsGHG - compar.pred1
compar['diff2'] = compar.obs_eng - compar.pred2
compar.head()

,obsGHG,pred1,obs_eng,pred2,diff1,diff2
1125,7.49,NaN,1074989.000,NaN,NaN,NaN
2101,27.83,NaN,817255.875,NaN,NaN,NaN
9,151.12,3794.196755,4573777.000,1.094836e+08,-3643.076755,-1.049098e+08
2492,22.33,NaN,588856.125,NaN,NaN,NaN
1929,40.01,NaN,1429855.750,NaN,NaN,NaN


In [445]:
with open('log_best_model.pkl', 'wb') as fileWriteStream:
    pickle.dump(log_model, fileWriteStream)

### shap

In [453]:
# calculate aic 
mse=mean_squared_error(y_test, preds_log)
def calculate_aic(n, mse, num_params):
	aic = n * log(mse) + 2 * num_params
	return aic
num_params = len(model.coef_) + 1
calculate_aic(len(y_test), mse,num_params)

AttributeError: 'dict' object has no attribute 'coef_'

In [ ]:
import matplotlib.pyplot as plt
f = plt.figure()
shap.summary_plot(rf_shap_values, X_test)
f.savefig("/summary_plot1.png", bbox_inches='tight', dpi=600)